### ROPE

In [4]:
import torch.nn as nn
import torch

# type(nn.Linear())

In [1]:
import torch

In [5]:
seq_len = 6
mask = torch.triu(torch.ones(seq_len, seq_len), diagonal=1)
mask

tensor([[0., 1., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1., 1.],
        [0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0.]])

In [7]:
mask = mask.masked_fill(mask == 1, float('-inf'))
mask

tensor([[0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0.]])

In [10]:
scores = torch.randn(seq_len,seq_len)
scores+mask

tensor([[ 0.2201,    -inf,    -inf,    -inf,    -inf,    -inf],
        [ 1.3764,  0.6220,    -inf,    -inf,    -inf,    -inf],
        [ 0.2803,  0.1554, -0.7288,    -inf,    -inf,    -inf],
        [ 0.3666, -0.3853,  0.4477,  1.3087,    -inf,    -inf],
        [-0.2056,  0.9088, -0.8399, -0.2532, -0.4475,    -inf],
        [ 0.1551,  0.0405, -0.5983,  0.3143,  0.9515,  0.0617]])

In [6]:
dim = 8
(torch.arange(0, dim, 2).float() / dim)

theta = 10_000

theta**0.2

6.309573444801933

In [2]:
import torch

dim = 8
theta = 10_000
freqs = 1.0 / (theta ** (torch.arange(0, dim, 2).float() / dim))
freqs


tensor([1.0000, 0.1000, 0.0100, 0.0010])

In [5]:
torch.arange(0, dim, 2).float() /dim



tensor([0.0000, 0.2500, 0.5000, 0.7500])

In [8]:
end = 128 #max sequence size
t = torch.arange(end, device=freqs.device)
t

tensor([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
         28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
         42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
         56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
         70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
         84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
         98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
        112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
        126, 127])

In [3]:
text = ['The', 'cat', 'sat', 'on', 'the', 'mat']
buf = text
print(text[0:3+1])
print(buf[1:4+1])


['The', 'cat', 'sat', 'on']
['cat', 'sat', 'on', 'the']


In [9]:
t[1]*freqs

tensor([1.0000, 0.1000, 0.0100, 0.0010])

In [10]:
freqs_u = torch.outer(t, freqs).float()
freqs_u[1], len(freqs_u)


(tensor([1.0000, 0.1000, 0.0100, 0.0010]), 128)

In [11]:
freqs_u.shape

torch.Size([128, 4])

In [12]:
x = torch.polar(torch.ones_like(freqs_u), freqs_u)  # complex64
x[2], len(x) #this is basically like generating angle for each pair of dim

(tensor([-0.4161+0.9093j,  0.9801+0.1987j,  0.9998+0.0200j,  1.0000+0.0020j]),
 128)

In [58]:
xq = torch.randn(128,4,8) # batchsize, seq_len, num_heads, head_dim
xk = torch.randn(xq.shape)

In [59]:
##apply rotatory embedding
xq_ = torch.view_as_complex(xq.float().reshape(*xq.shape[:-1], -1, 2)) # (Seq, N_Heads, Head_Dim) --> (Seq, N_Heads, Head_Dim // 2)
xk_ = torch.view_as_complex(xk.float().reshape(*xk.shape[:-1], -1, 2)) # (Seq, N_Heads, Head_Dim) --> (Seq, N_Heads, Head_Dim // 2)

In [60]:
xq_.shape

torch.Size([128, 4, 4])

In [61]:
freqs_cis = freqs_u[:, None, :]       # (L, 1, D/2)
freqs_cis

tensor([[[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00]],

        [[1.0000e+00, 1.0000e-01, 1.0000e-02, 1.0000e-03]],

        [[2.0000e+00, 2.0000e-01, 2.0000e-02, 2.0000e-03]],

        [[3.0000e+00, 3.0000e-01, 3.0000e-02, 3.0000e-03]],

        [[4.0000e+00, 4.0000e-01, 4.0000e-02, 4.0000e-03]],

        [[5.0000e+00, 5.0000e-01, 5.0000e-02, 5.0000e-03]],

        [[6.0000e+00, 6.0000e-01, 6.0000e-02, 6.0000e-03]],

        [[7.0000e+00, 7.0000e-01, 7.0000e-02, 7.0000e-03]],

        [[8.0000e+00, 8.0000e-01, 8.0000e-02, 8.0000e-03]],

        [[9.0000e+00, 9.0000e-01, 9.0000e-02, 9.0000e-03]],

        [[1.0000e+01, 1.0000e+00, 1.0000e-01, 1.0000e-02]],

        [[1.1000e+01, 1.1000e+00, 1.1000e-01, 1.1000e-02]],

        [[1.2000e+01, 1.2000e+00, 1.2000e-01, 1.2000e-02]],

        [[1.3000e+01, 1.3000e+00, 1.3000e-01, 1.3000e-02]],

        [[1.4000e+01, 1.4000e+00, 1.4000e-01, 1.4000e-02]],

        [[1.5000e+01, 1.5000e+00, 1.5000e-01, 1.5000e-02]],

        [[1.6000e+01, 1.

In [62]:
xq_.shape,freqs_cis.shape

(torch.Size([128, 4, 4]), torch.Size([128, 1, 4]))

In [73]:
torch.view_as_real(xq_[0]*freqs_cis[0]).flatten(2).shape

torch.Size([4, 4, 2])

In [65]:
torch.view_as_real(xq_ * freqs_cis)

tensor([[[[ 0.0000e+00, -0.0000e+00],
          [ 0.0000e+00,  0.0000e+00],
          [ 0.0000e+00, -0.0000e+00],
          [ 0.0000e+00,  0.0000e+00]],

         [[-0.0000e+00,  0.0000e+00],
          [ 0.0000e+00, -0.0000e+00],
          [ 0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00]],

         [[ 0.0000e+00,  0.0000e+00],
          [-0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00]],

         [[-0.0000e+00,  0.0000e+00],
          [ 0.0000e+00, -0.0000e+00],
          [-0.0000e+00,  0.0000e+00],
          [-0.0000e+00,  0.0000e+00]]],


        [[[-4.4467e-01,  1.2905e+00],
          [ 9.8830e-02, -1.0726e-01],
          [ 6.8933e-03, -6.3250e-03],
          [-2.2125e-04, -1.2586e-04]],

         [[-9.7308e-01,  2.4362e-01],
          [ 6.7896e-02,  5.7896e-02],
          [ 2.2173e-03, -1.1317e-02],
          [-1.2682e-03, -1.5833e-04]],

         [[ 4.7539e-01,  9.4646e-01],
          [ 3.7943e-02,  8.8356e-03]

In [64]:
xq_out = torch.view_as_real(xq_ * freqs_cis).flatten(2)
xk_out = torch.view_as_real(xk_ * freqs_cis).flatten(2)


xq_out.shape

torch.Size([128, 4, 8])

### Attention

In [1]:
import torch

torch.randn(1,2,3,2,7)





/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/torch/_subclasses/functional_tensor.py:276: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:81.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


tensor([[[[[-0.4533, -0.3675, -0.1270,  1.1463, -0.2604,  0.0926,  0.9946],
           [-0.4162,  1.8222,  0.4044, -0.2206,  0.1385, -2.9662,  0.1371]],

          [[-1.2895,  0.1460,  0.7678,  1.8731,  1.3455,  1.1769,  0.5677],
           [ 1.0320, -0.2058, -0.0978, -1.6889, -0.4842,  1.1899, -0.6999]],

          [[-2.3260, -1.3848, -0.0641, -1.5364,  0.2893,  0.1010, -1.7162],
           [ 1.7862, -0.0540, -0.4318, -0.2321,  0.1171, -0.7251, -1.6531]]],


         [[[ 0.4898, -0.2573, -0.3233,  0.2182,  0.2094,  0.6430, -0.7222],
           [-0.0053,  0.4473, -1.0618, -0.5134,  0.1241, -1.7960, -0.9455]],

          [[-0.0352, -1.6997, -0.2395, -0.4203,  0.2276, -0.6762,  1.9303],
           [-0.3389, -0.2476,  1.1896, -0.5112, -0.0305,  0.4175, -0.9804]],

          [[-0.5624,  0.6293, -0.4875, -1.2859,  0.8312,  0.3315,  0.0638],
           [-0.1799,  1.4999, -0.0130,  0.3338,  0.5623,  1.0163, -0.3292]]]]])

In [4]:
torch.randn(1,3,2,4)

tensor([[[[-0.5133,  1.0696, -1.0343,  0.7257],
          [-0.1375,  0.6273, -1.0861, -0.0903]],

         [[-1.2342,  1.5384, -2.2976,  1.0724],
          [-1.2737, -0.6474,  0.0996,  1.6319]],

         [[-0.7280,  1.4553, -1.0420,  0.6409],
          [-0.5856, -0.5049, -2.0347,  0.0171]]]])

In [7]:
import torch
mask=torch.tril(torch.ones(5,5,device='mps'))
mask=mask[-1]
# mask=mask.view(1,1,1,window_size).expand(batch_size,num_heads,seq_len,window_size)

In [8]:
mask

tensor([1., 1., 1., 1., 1.], device='mps:0')

In [9]:
mask=mask.view(1,1,1,5).expand(1,4,3,5)
mask

tensor([[[[1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.]],

         [[1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.]],

         [[1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.]],

         [[1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.]]]], device='mps:0')

In [2]:
%uv pip install torchinfo tqdm tiktoken


Using Python 3.13.3 environment at: /Library/Frameworks/Python.framework/Versions/3.13
Resolved 9 packages in 233ms                                         
⠙ Preparing packages... (0/3)                                                   
⠙ Preparing packages... (0/3)----     0 B/984.25 KiB                    
regex      ------------------------------     0 B/277.95 KiB
⠙ Preparing packages... (0/3)----     0 B/984.25 KiB                    
regex      ------------------------------ 16.00 KiB/277.95 KiB
⠙ Preparing packages... (0/3)----     0 B/984.25 KiB                    
regex      ------------------------------ 16.00 KiB/277.95 KiB
⠙ Preparing packages... (0/3)---- 8.74 KiB/984.25 KiB                   
regex      ------------------------------ 32.00 KiB/277.95 KiB
⠙ Preparing packages... (0/3)---- 8.74 KiB/984.25 KiB                   
regex      ------------------------------ 48.00 KiB/277.95 KiB
⠙ Preparing packages... (0/3)---- 8.74 KiB/984.25 KiB                   
regex      

In [8]:
from torchinfo import summary
from model import minimixtral
from config import ModelArgs, MoeArgs


args = ModelArgs(vocab_size=50257,moe=MoeArgs())
mini_mistral = minimixtral(args)

# summary(mini_mistral,input_size=(4,256,512))


In [9]:
def count_parameters(model):
    total = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Total parameters: {total:,}")
    print(f"Trainable parameters: {trainable:,}")

# Example usage
count_parameters(mini_mistral)

Total parameters: 195,433,105
Trainable parameters: 195,433,105


In [17]:
import torch, tiktoken, glob
from torch.serialization import add_safe_globals
from config import ModelArgs
from model  import minimixtral           # your network
from pathlib import Path
# ── allow custom class in safe unpickler ───────────────────────
add_safe_globals([ModelArgs])

# ckpt_path = sorted(glob.glob("ckpts/*.pt"))[0]
ckpt_path = Path('ckpts/best_step124.pt')

print("Loading", ckpt_path)

# 1) load on CPU
ckpt = torch.load(ckpt_path, map_location="cpu", weights_only=False)
args = ckpt["args"]

# 2) make constructor stay on CPU
args.device = "cpu"                      # ← critical line

model = minimixtral(args)                # no CUDA tensors yet
model.load_state_dict(ckpt["model"], strict=False)

# 3) optionally move to GPU once, safely
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to("cpu")

model.eval()

# 4) ready to generate
tok  = tiktoken.get_encoding("gpt2")
EOT  = tok._special_tokens["<|endoftext|>"]

def encode(t): return tok.encode_ordinary(t)
def decode(i): return tok.decode(i)

@torch.no_grad()
def complete(prompt, max_new=120, temp=0.8, top_p=0.9):
    dv  = next(model.parameters()).device
    ids = torch.tensor(encode(prompt)+[EOT], device=dv)

    for _ in range(max_new):
        logits = model(ids.unsqueeze(0), 0)[0, -1] / temp
        probs  = torch.softmax(logits, -1)

        sorted_idx = torch.argsort(probs, descending=True)
        cumsum     = torch.cumsum(probs[sorted_idx], 0)
        keep       = cumsum <= top_p
        keep[0]    = True
        probs[sorted_idx[~keep]] = 0
        probs /= probs.sum()

        next_id = torch.multinomial(probs, 1)
        if next_id.item() == EOT:
            break
        ids = torch.cat([ids, next_id])
    return decode(ids.tolist()[len(encode(prompt)):])

# quick demo
for _ in range(5):
    print("\n--- completion ---\n" +
        complete("Long time ago, ") + "\n")





Loading ckpts/best_step124.pt



--- completion ---
<|endoftext|> pharmaceutical far dice time, but he Then oldOK, but he could help saw, but he had a big, but he had a big, but he happy thatGoode, but they it doing on, but she could matching she desk, Mom had a big, laughed you noticed wand, he had you out of is this, she saw a time, for, she looked finding had a adventure, that he she garden, built, that you help, You of play his mom hands, " looked he hugs sound, child, but he would, she saw he Mia, hugged, I are


--- completion ---
<|endoftext|>Once falling, but be look they picked, but it too had a big, but


--- completion ---
<|endoftext|>Tom, but he said, but she island, but it, but was so, ruined, thought, has, Lily, there, snake, they, you, she, they you, lots of would, Lily, Lily, Lily, of, she towards, Lily, they, Lily, Lily, she cooking, you morning,, Lily, chocolate, all, I, she, she had that, she, Lily, stopped, opened, he an, don, they were, he heard, Lily, she, until, she sw, she, heNo, Lily, she wh

In [ ]:
--- completion ---
<|endoftext|> painted down on, but he saw theyily they Thank, Toby you want Jane, but he also Mom, but he looked One, but it, but she open are walked, but he with triedYes, but she had after again, but he had of bounce, but he out what couldn't wear, but they ice garden, but, again going out, but he you too, sir, Lily table Can, she bottle, Lily so, or, on, we are, garden, she had you have, Lily, she hadNo, she remembered, Lily around, she who, Mom

In [ ]:
--- completion ---
<|endoftext|>Once upon a time there were two dragons they were so happy that he had name you only, but she could day, but he had a! He saw happened said that of talk toy heWhen had a big cake that he had a only some that she had a big, and be finished old am youHello off when will time sheOne day, but he tried you who lunch, he had a sky what that she took a big that she looked be very happy that he had a big you so they had a big soft, Lily that tried looked long is for. He would you can is Bob

In [18]:
%pip install huggingface_hub


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import torch
import torch.nn as nn
from huggingface_hub import PyTorchModelHubMixin


from huggingface_hub import HfApi

api = HfApi()
repo_id = "rajtiwariee/my-manually-uploaded-model"
api.create_repo(repo_id=repo_id, private=False, exist_ok=True)

In [24]:
from huggingface_hub import HfApi
import os

HF_TOKEN = os.getenv("HF_TOKEN")          # or paste the string here
api      = HfApi(token=HF_TOKEN)

repo_id = "rajtiwariee/minimixtral"       # <user>/<repo_name>

# 1️⃣ create the repo once (does nothing if it already exists)
api.create_repo(repo_id=repo_id,
                repo_type="model",
                private=False,            # ↙ change if you want a private repo
                exist_ok=True)

# 2️⃣ now push your checkpoint folder
api.upload_folder(
    folder_path="ckpts/best",             # local directory
    repo_id=repo_id,
    repo_type="model"
)
print("✅ upload done!")


minimixtral.pt:   0%|          | 0.00/1.05G [00:00<?, ?B/s]

✅ upload done!
